# Data Parsing

Here we will try to parse the Aminer V12 json file into dataframe.

In [ ]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

Defining the data file path

In [ ]:
data_file = '../Data/dblp.v12.json'

Using `yield` to load the JSON file in a loop to prevent Python memory issues if JSON is loaded directly

In [ ]:
def get_metadata():
    with open(data_file, 'r', encoding='utf-8') as f:
        next(f)                                         #skip the first line
        for line in f:
            yield line
metadata = get_metadata()

In [ ]:
paper_list = []
counter = 0
for paper in metadata:   
    paper = paper.strip(',')       # takes care of the errors with ',' in begining of the line
    if paper == None or paper == '' or paper == '[' or paper == ']' or len(paper) == 0:
        print('I got a null or empty string value for paper in a file')
    else:       
        try:
            paper_dict = json.loads(paper)
            year = int(paper_dict.get('year'))
            to_pop = ['page_start', 'page_end','doc_type','publisher','volume','issue','doi','alias_ids','title','indexed_abstract','fos','venue','authors']
            for i in to_pop:
                try:
                    paper_dict.pop(i)
                except:
                    pass 
            if year <= 2010:
              paper_list.append(paper_dict)
              counter += 1
            if (counter % 100000 == 0):
                print(counter)
                print(paper_dict)
        except:
            pass 

In [ ]:
# checking the length of data found
len(paper_list)

In [ ]:
# now add the list into dataframe
papers_df = pd.DataFrame(paper_list)

In [ ]:
papers_df.info()
papers_df.head()

Let's Write the paper dataFrame into pickle and csv for later use

In [ ]:
path = '../Data/papers_0_2010.'
papers_df.to_pickle(path + 'pkl')

In [ ]:
papers_df.to_csv(path + 'csv')